In [1]:
from collections import defaultdict
from HW3Utils import *
from math import log

In [2]:
def train(messages):
    classes_count = defaultdict(lambda:0)
    words_count = defaultdict(lambda:0)

    total_words = 0
    for message in messages:
        label = 0 if message.isSpam() else 1
        classes_count[label] += len(message.getAllWords())
        total_words += len(message.getAllWords())
        for word in message.getAllWords():
            words_count[label, word] += 1
    for c, word in words_count:
        words_count[c, word] /= ((0 if words_count.get((0, word)) is None else words_count.get((0, word))) + \
                                 (0 if words_count.get((1, word)) is None else words_count.get((1, word))))
    for c in classes_count:
        classes_count[c] /= total_words
    return classes_count, words_count

In [7]:
def classify(probabilities, message):
    classes_count, words_count = probabilities
    result = [0] * len(classes_count)
    for i in range(len(classes_count)):
        result[i] = log(classes_count[i])
        for word in message.getAllWords():
            if words_count[i, word] != 0:
                result[i] += log(words_count[i, word])
    return 0 if log(result[0] / result[1]) > 0 else 1  # for fn minimalization 0.5 is ok

In [8]:
messages = getData()

averageAccuracy = 0
for i in range(10):
    train_messages = []
    test_messages = []
    for message in messages:
        if message.getFold() == i:
            test_messages.append(message)
        else:
            train_messages.append(message)
    probabilities = train(train_messages)
    answ = []
    for message in test_messages:
        answ.append(classify(probabilities, message))
    accuracy, fn = f1_score(answ, [msg.isSpam() for msg in test_messages])
    averageAccuracy += accuracy
    print(accuracy, fn)
    
averageAccuracy /= 10
print('Average:', averageAccuracy)

0.7328244274809159 0
0.6808510638297872 0
0.7328244274809159 0
0.6906474820143885 0
0.7111111111111111 0
0.7022900763358779 2
0.7230769230769231 1
0.7328244274809159 0
0.7441860465116279 0
0.696969696969697 2
Average: 0.7147605682292161
